In [1]:
import sys
sys.path.append('..')

import os
import shutil
import requests
import dataclasses
from pathlib import Path
import json

import dotenv

import fiona
from pyproj import Transformer
from shapely.ops import transform, unary_union
from shapely.geometry import shape, mapping

dotenv.load_dotenv()

True

In [2]:
from typing import List


@dataclasses.dataclass
class Source:
    urls: List[str]
    kind: str
    # layer: str

In [3]:
SHAPEFILE_EXTS = ['shp', 'shx', 'dbf', 'prj']

SOURCES = [
    Source(['https://s3.us-east-1.wasabisys.com/opentreebase-public/source/beer-sheva/open-data/2022/trees.geojson'], 'trees'),
    Source(['https://s3.us-east-1.wasabisys.com/opentreebase-public/source/jerusalem/surveys/2022/tree_survey.' + x for x in SHAPEFILE_EXTS], 'trees'),
    Source(['https://s3.us-east-1.wasabisys.com/opentreebase-public/source/beer-sheva/canopies/2022/Trees_Beer_Sheva.' + x for x in SHAPEFILE_EXTS], 'canopies'),
]

In [11]:
TEMPDIR = Path('temp')
OUTPUTS = {}
for kind in ['trees', 'canopies']:
    outfilename = TEMPDIR / (kind + '.geojson')
    OUTPUTS[kind] = outfilename
    with outfilename.open('w') as outfile:
        outfile.write('{"type": "FeatureCollection", "features": [')
        first = True

        for source in SOURCES:
            if source.kind != kind:
                continue

            parts = source.urls[0].split('/')[4:7]
            slug = '-'.join(parts)

            os.makedirs(TEMPDIR / slug, exist_ok=True)

            # Download the file into temp file with the same extension
            source_file = None
            for url in source.urls:
                tempfile = (TEMPDIR / slug / slug).with_suffix(Path(url).suffix)
                if not tempfile.exists():
                    with tempfile.open('wb') as temp:
                        print(slug, 'DOWNLOADING', url)
                        r = requests.get(url, stream=True).raw
                        shutil.copyfileobj(r, temp)
                if source_file is None:
                    source_file = tempfile

            # Open the file with fiona
            layer = fiona.listlayers(source_file)[0]
            print(slug, 'LAYER', layer)
            with fiona.open(source_file, layername=layer) as collection:
                print(slug, 'CRS', collection.crs)
                transformer = None
                if collection.crs['init'] != 'epsg:4326':
                    transformer = Transformer.from_crs(collection.crs['init'], 'epsg:4326', always_xy=True)
                    
                for item in collection.filter():
                    if item['geometry'] is None:
                        continue
                    geometry = item['geometry']
                    if transformer is not None:
                        geometry = mapping(transform(transformer.transform, shape(geometry)))
                    if first:
                        first = False
                    else:
                        outfile.write(',')
                    outfile.write(json.dumps(dict(
                        type='Feature',
                        properties={},
                        geometry=geometry,
                    )))
        outfile.write(']}')


source-beer-sheva-open-data LAYER source-beer-sheva-open-data
source-beer-sheva-open-data CRS {'init': 'epsg:4326'}
source-jerusalem-surveys LAYER source-jerusalem-surveys
source-jerusalem-surveys CRS {'init': 'epsg:2039'}
source-beer-sheva-canopies LAYER source-beer-sheva-canopies
source-beer-sheva-canopies CRS {'init': 'epsg:2039'}


In [ ]:
from notebooks.mapbox_utils import run_tippecanoe, upload_tileset

for kind, filename in OUTPUTS.items():
    print(kind, filename)
    mbtiles_filename = str(filename.with_suffix('.mbtiles'))
    if run_tippecanoe('-z15', str(filename), '-o', mbtiles_filename, 
                      '-l', kind):
        print('SUCCESS')
        upload_tileset(mbtiles_filename, 'treebase.' + kind, kind.title() + ' Data')
